In [1]:
import numpy as np
import scipy as sp
import random
import cmath
import math

# Define our buildDataSet function here!

In [2]:
def buildDataSet(max_amplitude,min_sparsity,max_sparsity,vector_size,data_set_size):
    sparse_data = np.zeros((vector_size,data_set_size)) # Initialize the sparse_data matrix

    # Iterate over the columns of the sparse_data matrix to define the data samples
    for i in range(data_set_size):
        sparsity = random.randint(min_sparsity,max_sparsity)
        indices = random.sample(range(vector_size),sparsity)
        amps = random.sample(range(max_amplitude),sparsity)
        sparse_data[indices,i] = amps
    
    # Define the DFT matrix and multiply our spare_data vectors with it to find dense data
    DFT = sp.linalg.dft(vector_size)/np.sqrt(vector_size)
    dense_data = DFT@sparse_data
    return dense_data,sparse_data

## Build the dataset

In [85]:
max_amplitude = 10
min_sparsity = 3
max_sparsity = 5
vector_size = 4*100
data_set_size = 1000
dense_data, sparse_data = buildDataSet(max_amplitude,min_sparsity,max_sparsity,vector_size,data_set_size)    

## Test the dataset

In [5]:
DFT = sp.linalg.dft(vector_size)/np.sqrt(vector_size)
iDFT = DFT.conj().T

iDFT@dense_data

[[-8.32383925e-17-5.30495957e-15j -3.75908315e-15+1.55827220e-15j
  -5.67563560e-15-1.27511087e-14j ... -3.91905967e-16-5.59725313e-15j
   1.04465676e-14-1.07786490e-14j -4.08380551e-15+1.53525282e-14j]
 [-4.32645966e-15-2.81770807e-15j -5.86865948e-15-3.66864353e-15j
  -3.03794343e-15+1.18800780e-14j ... -5.78088144e-15-1.58742081e-14j
  -5.20309979e-15-1.46985806e-14j  1.69142069e-14-2.48480443e-14j]
 [-3.93647314e-15+2.13124858e-15j  1.54628366e-15+2.77167802e-15j
   1.08119353e-15-1.26130583e-15j ... -2.32706105e-14+1.29636896e-14j
  -1.14119808e-14-1.77664643e-15j -1.68235505e-14-4.48146349e-15j]
 ...
 [-2.57483018e-15-2.00013029e-15j -1.01230986e-14+4.56469969e-15j
  -3.94736216e-15+1.61422553e-14j ...  9.22216633e-15+1.74251633e-14j
  -1.91337725e-14-1.24438804e-14j  3.18320655e-15-1.08903105e-14j]
 [-1.11613546e-14-7.53323639e-16j -7.43453566e-15+6.46876808e-15j
   3.93988894e-15-7.05815539e-17j ... -4.74455272e-15+2.32939712e-15j
  -1.59863324e-14+1.48105404e-14j -2.92970740e-

From the above results, we can see that our vectors are very sparse if we take the IDFT

## Setting up the dataset for Pytorch

In [82]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split

X = np.concatenate((dense_data.real, dense_data.imag), axis=0).T
Y = np.concatenate((sparse_data.real, sparse_data.imag), axis=0).T

X_tensor = torch.tensor(X,dtype=torch.float)
Y_tensor = torch.tensor(Y,dtype=torch.float)
dataset = TensorDataset(X_tensor,Y_tensor)

dataloader = DataLoader(dataset,batch_size = 200,shuffle = True)

## Setting up the neural network

In [86]:
# class ExponentialComplexLinear(nn.Module):
#     def __init__(self, in_features, out_features):
#         super(ExponentialComplexLinear, self).__init__()
#
#         # Trainable phase angles (real-valued), this is where we are training the q-values in the context of the exercise
#         self.phases = nn.Parameter(torch.randn(out_features, in_features) * 0.1)  # Small random initialization
#
#     def forward(self, x):
#         # Compute our matrix
#         W = torch.exp(1j * self.phases)  # Enforces |W| = 1
#
#         # Complex matrix multiplication
#         return torch.matmul(x, W.t())

class ComplexDecoder(nn.Module):
    def __init__(self, encoding_dim, output_dim):
        super(ComplexDecoder, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(encoding_dim,400),
            nn.ReLU(),
            nn.Linear(400,400),
            nn.ReLU(),
            nn.Linear(400,output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

class ComplexReLU(nn.Module):
    def forward(self, x):
        return torch.complex(F.relu(x.real), F.relu(x.imag))  # Apply ReLU separately

# class LearnedAutoencoder(nn.Module):
#     def __init__(self, input_dim, encoding_dim):
#         super(LearnedAutoencoder, self).__init__()
#         # Encoder: Maps from input_dim to a lower-dimensional complex encoding space.
#         self.encoder = ExponentialComplexLinear(input_dim, encoding_dim)
#
#         # Decoder: Maps from the encoding space back to the original input dimension.
#         self.decoder = ComplexDecoder(encoding_dim, input_dim)
#
#     def forward(self, x):
#         encoded = self.encoder(x)
#         decoded = self.decoder(encoded)
#         return decoded



## Training the model

In [87]:
# Define the size of our "measurement" vector as encoding_dim. This needs to be larger than the sparsity of our matrix

encoding_dim = max_sparsity + 1

# Initialize model
model = ComplexDecoder(vector_size, encoding_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

def complex_mse_loss(input, target):
    return F.mse_loss(input.real, target.real) + F.mse_loss(input.imag, target.imag)

# Training loop
for epoch in range(1000):
    for batch in dataloader:
        inputs, targets = batch  # Unpack the tuple
        optimizer.zero_grad()
        output = model(inputs)
        loss = complex_mse_loss(output, targets)  # Custom loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.6f}")

/tmp/ipykernel_4873/636337709.py:11: UserWarning: Using a target size (torch.Size([200, 400])) that is different to the input size (torch.Size([200, 6])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input.real, target.real) + F.mse_loss(input.imag, target.imag)


RuntimeError: The size of tensor a (6) must match the size of tensor b (400) at non-singleton dimension 1